# Memoria EDA "Voto por renta, el caso de Gipuzkoa 2016-2019"

El objetivo de este proyecto de EDA *(Exploratory Data Analisys)* es analizar los datos de participación y renta al nivel censal más pequeño posible (que en españa es la mesa electoral), y analizar la correlación (o su ausencia) entre la renta media y la participación en el proceso electoral.

El análisis partirá de una hipótesis primaria, "a mayor renta, mayor participación electoral", que será verificada con los datos de los procesos electorales de las elecciones al congreso de los diputados de 2016 y 2019, individualmente.

Tras la confirmación o no de la hipótesis, se analizará el comportamiento de aquellas secciones censales que, por motivos de cambio macroeconómico, varían significativamente su nivel de renta, buscando si esto conlleva un cambio proporcional en la participación.

## Obtención de datos

El EDA usará dos fuentes primarias de datos on-line, para los dos parámetros principales que necesita cruzar:

+ Renta: [Instituto Nacional de Estadística](https://www.ine.es/jaxiT3/Tabla.htm?t=31007&L=0)
    
+ Participación: [Ministerio del Interior](https://infoelectoral.interior.gob.es/opencms/es/elecciones-celebradas/area-de-descargas/)